In [1]:
import tensorflow as tf
import numpy as np

In [20]:
data_x = np.loadtxt('data/input_x.csv', dtype=float)
data_y = np.loadtxt('data/input_y.csv', dtype=float)

data_x_test = np.loadtxt('data/input_x_test.csv', dtype=float)
data_y_test = np.loadtxt('data/input_y_test.csv', dtype=float)

print(data_x.shape, data_y.shape, data_x_test.shape, data_y_test.shape)

(141, 16) (141, 4) (42, 16) (42, 4)


In [23]:
print(data_x[5], data_y[5])

[0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1.] [0. 0. 0. 1.]


In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(16,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 340
Trainable params: 340
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(data_x, data_y, epochs=200)


Epoch 1/200
141/141 [==============================] - 0s 761us/sample - loss: 0.0314 - acc: 0.9504
Epoch 2/200
141/141 [==============================] - 0s 82us/sample - loss: 0.0255 - acc: 0.9787
Epoch 3/200
141/141 [==============================] - 0s 69us/sample - loss: 0.0262 - acc: 0.9574
Epoch 4/200
141/141 [==============================] - 0s 77us/sample - loss: 0.0274 - acc: 0.9504
Epoch 5/200
141/141 [==============================] - 0s 71us/sample - loss: 0.0316 - acc: 0.9433
Epoch 6/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0209 - acc: 0.9858
Epoch 7/200
141/141 [==============================] - 0s 63us/sample - loss: 0.0234 - acc: 0.9858
Epoch 8/200
141/141 [==============================] - 0s 68us/sample - loss: 0.0248 - acc: 0.9716
Epoch 9/200
141/141 [==============================] - 0s 58us/sample - loss: 0.0237 - acc: 0.9504
Epoch 10/200
141/141 [==============================] - 0s 84us/sample - loss: 0.0234 - acc: 0.9787
Epoch 11

141/141 [==============================] - 0s 60us/sample - loss: 0.0085 - acc: 0.9929
Epoch 84/200
141/141 [==============================] - 0s 62us/sample - loss: 0.0093 - acc: 0.9929
Epoch 85/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0154 - acc: 0.9787
Epoch 86/200
141/141 [==============================] - 0s 61us/sample - loss: 0.0094 - acc: 0.9929
Epoch 87/200
141/141 [==============================] - 0s 55us/sample - loss: 0.0106 - acc: 0.9929
Epoch 88/200
141/141 [==============================] - 0s 64us/sample - loss: 0.0106 - acc: 0.9929
Epoch 89/200
141/141 [==============================] - 0s 58us/sample - loss: 0.0078 - acc: 0.9929
Epoch 90/200
141/141 [==============================] - 0s 71us/sample - loss: 0.0095 - acc: 0.9929
Epoch 91/200
141/141 [==============================] - 0s 57us/sample - loss: 0.0163 - acc: 0.9574
Epoch 92/200
141/141 [==============================] - 0s 63us/sample - loss: 0.0108 - acc: 0.9929
Epoch 93/200


141/141 [==============================] - 0s 66us/sample - loss: 0.0086 - acc: 0.9787
Epoch 165/200
141/141 [==============================] - 0s 56us/sample - loss: 0.0061 - acc: 1.0000
Epoch 166/200
141/141 [==============================] - 0s 55us/sample - loss: 0.0080 - acc: 0.9858
Epoch 167/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0033 - acc: 1.0000
Epoch 168/200
141/141 [==============================] - 0s 63us/sample - loss: 0.0074 - acc: 1.0000
Epoch 169/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0053 - acc: 1.0000
Epoch 170/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0050 - acc: 0.9929
Epoch 171/200
141/141 [==============================] - 0s 53us/sample - loss: 0.0049 - acc: 0.9929
Epoch 172/200
141/141 [==============================] - 0s 57us/sample - loss: 0.0041 - acc: 1.0000
Epoch 173/200
141/141 [==============================] - 0s 60us/sample - loss: 0.0093 - acc: 0.9858
Epoc

In [39]:
model.evaluate(data_x_test, data_y_test)

42/42 [==============================] - 0s 1ms/sample - loss: 0.0021 - acc: 1.0000


[0.0020744032226502895, 1.0]

In [40]:
predictions = model.predict(data_x_test)

In [41]:
command_classes = ['count', 'color', 'focus', 'no_op']


In [45]:
[command_classes[np.argmax(p)] for p in predictions]

['count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus',
 'count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus',
 'count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus',
 'count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus',
 'count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus',
 'count',
 'count',
 'color',
 'color',
 'focus',
 'focus',
 'focus']

In [46]:
model.save('data/sf_nn.h5')